In [1]:
import pandas as pd

In [2]:
csv_files_dict={"2013":"Games_2013.csv","2014":"Games_2014.csv","2015":"Games_2015.csv","2016":"Games_2016.csv","2017":"Games_2017.csv","2018":"Games_2018.csv","2019":"Games_2019.csv","2020":"Games_2020.csv","2021":"Games_2021.csv","2022":"Games_2022.csv","2023":"Games_2023.csv","2024":"Games_2024.csv"}
csv_files_dict

{'2013': 'Games_2013.csv',
 '2014': 'Games_2014.csv',
 '2015': 'Games_2015.csv',
 '2016': 'Games_2016.csv',
 '2017': 'Games_2017.csv',
 '2018': 'Games_2018.csv',
 '2019': 'Games_2019.csv',
 '2020': 'Games_2020.csv',
 '2021': 'Games_2021.csv',
 '2022': 'Games_2022.csv',
 '2023': 'Games_2023.csv',
 '2024': 'Games_2024.csv'}

In [3]:
df_list=[]

In [4]:
def make_df_func(yr,path):
    path_first="../Project 2/"
    path_second=path
    full_path=path_first + path_second
    df=pd.read_csv(full_path)
    df.rename(columns={"Unnamed: 4":"H/A"},inplace=True)
    df.drop("Unnamed: 2",axis=1,inplace=True)
    df["H/A"].fillna("H",inplace=True)
    df["Year"]=yr
    df_list.append(df)
    return df
    

In [5]:
for key,value in csv_files_dict.items():
    result=make_df_func(key,value)

mega_df=pd.concat(df_list,ignore_index=True)
mega_df

,Gm#,Date,Tm,H/A,Opp,W/L,R,RA,Inn,W-L,...,Win,Loss,Save,Time,D/N,Attendance,cLI,Streak,Orig. Scheduled,Year
0,1,Monday Apr 1,COL,@,MIL,L-wo,4,5,10.0,0-1,...,Henderson,Ottavino,NaN,3:13,D,45781,0.93,-,NaN,2013
1,2,Tuesday Apr 2,COL,@,MIL,W,8,4,NaN,1-1,...,Escalona,Gonzalez,Betancourt,3:22,N,24753,0.93,+,NaN,2013
2,3,Wednesday Apr 3,COL,@,MIL,W,7,3,NaN,2-1,...,Nicasio,Peralta,NaN,3:31,N,25766,0.95,++,NaN,2013
3,4,Friday Apr 5,COL,H,SDP,W,5,2,NaN,3-1,...,Francis,Marquis,Betancourt,2:45,D,49077,1.08,+++,NaN,2013
4,5,Saturday Apr 6,COL,H,SDP,W,6,3,NaN,4-1,...,Garland,Ross,Betancourt,2:54,N,31133,1.12,++++,NaN,2013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1837,158,Wednesday Sep 25,COL,H,STL,8:40 pm,8:40 pm,8:40 pm,Game Preview and Matchups,Game Preview and Matchups,...,Game Preview and Matchups,Game Preview and Matchups,Game Preview and Matchups,Game Preview and Matchups,Game Preview and Matchups,Game Preview and Matchups,NaN,NaN,NaN,2024
1838,159,Thursday Sep 26,COL,H,STL,3:10 pm,3:10 pm,3:10 pm,Game Preview and Matchups,Game Preview and Matchups,...,Game Preview and Matchups,Game Preview and Matchups,Game Preview and Matchups,Game Preview and Matchups,Game Preview and Matchups,Game Preview and Matchups,NaN,NaN,NaN,2024
1839,160,Friday Sep 27,COL,H,LAD,8:10 pm,8:10 pm,8:10 pm,Game Preview and Matchups,Game Preview and Matchups,...,Game Preview and Matchups,Game Preview and Matchups,Game Preview and Matchups,Game Preview and Matchups,Game Preview and Matchups,Game Preview and Matchups,NaN,NaN,NaN,2024
1840,161,Saturday Sep 28,COL,H,LAD,8:10 pm,8:10 pm,8:10 pm,Game Preview and Matchups,Game Preview and Matchups,...,Game Preview and Matchups,Game Preview and Matchups,Game Preview and Matchups,Game Preview and Matchups,Game Preview and Matchups,Game Preview and Matchups,NaN,NaN,NaN,2024


In [17]:
df = pd.read_csv("../Project 2/Games_2024.csv",index_col="Gm#")
df.head()

,Date,Unnamed: 2,Tm,Unnamed: 4,Opp,W/L,R,RA,Inn,W-L,...,GB,Win,Loss,Save,Time,D/N,Attendance,cLI,Streak,Orig. Scheduled
Gm#,,,,,,,,,,,,,,,,,,,,,
1,Thursday Mar 28,boxscore,COL,@,ARI,L,1,16,NaN,0-1,...,1.0,Gallen,Freeland,NaN,2:37,N,49011,1.00,-,NaN
2,Friday Mar 29,boxscore,COL,@,ARI,L,3,7,NaN,0-2,...,2.0,Kelly,Quantrill,NaN,2:36,N,29480,0.96,--,NaN
3,Saturday Mar 30,boxscore,COL,@,ARI,W,9,4,NaN,1-2,...,1.0,Lambert,Henry,NaN,3:17,N,32133,0.92,+,NaN
4,Sunday Mar 31,boxscore,COL,@,ARI,L,1,5,NaN,1-3,...,2.0,Pfaadt,Feltner,NaN,2:22,D,21824,0.97,-,NaN
5,Monday Apr 1,boxscore,COL,@,CHC,L,0,5,NaN,1-4,...,3.0,Imanaga,Hudson,NaN,2:26,D,40072,0.85,--,NaN


In [18]:
df.rename(columns={"Unnamed: 4":"H/A"},inplace=True)
df.drop("Unnamed: 2",axis=1,inplace=True)
df.head(10)

,Date,Tm,H/A,Opp,W/L,R,RA,Inn,W-L,Rank,GB,Win,Loss,Save,Time,D/N,Attendance,cLI,Streak,Orig. Scheduled
Gm#,,,,,,,,,,,,,,,,,,,,
1,Thursday Mar 28,COL,@,ARI,L,1,16,NaN,0-1,4,1.0,Gallen,Freeland,NaN,2:37,N,49011,1.00,-,NaN
2,Friday Mar 29,COL,@,ARI,L,3,7,NaN,0-2,5,2.0,Kelly,Quantrill,NaN,2:36,N,29480,0.96,--,NaN
3,Saturday Mar 30,COL,@,ARI,W,9,4,NaN,1-2,4,1.0,Lambert,Henry,NaN,3:17,N,32133,0.92,+,NaN
4,Sunday Mar 31,COL,@,ARI,L,1,5,NaN,1-3,5,2.0,Pfaadt,Feltner,NaN,2:22,D,21824,0.97,-,NaN
5,Monday Apr 1,COL,@,CHC,L,0,5,NaN,1-4,5,3.0,Imanaga,Hudson,NaN,2:26,D,40072,0.85,--,NaN
6,Tuesday Apr 2,COL,@,CHC,L,2,12,NaN,1-5,5,4.0,Assad,Freeland,NaN,2:37,N,26555,0.83,---,NaN
7,Wednesday Apr 3,COL,@,CHC,L,8,9,NaN,1-6,5,5.0,Alzolay,Mears,NaN,3:00,N,25900,0.81,----,NaN
8,Friday Apr 5,COL,NaN,TBR,W-wo,10,7,NaN,2-6,5,4.0,Beeks,Fairbanks,NaN,3:10,D,48399,0.72,+,NaN
9,Saturday Apr 6,COL,NaN,TBR,L,6,8,NaN,2-7,5,5.0,Cleavinger,Beeks,Poche,3:03,N,29238,0.71,-,NaN


In [20]:
df["H/A"].fillna("H",inplace=True)
df.tail()

,Date,Tm,H/A,Opp,W/L,R,RA,Inn,W-L,Rank,GB,Win,Loss,Save,Time,D/N,Attendance,cLI,Streak,Orig. Scheduled
Gm#,,,,,,,,,,,,,,,,,,,,
158,Wednesday Sep 25,COL,H,STL,8:40 pm,8:40 pm,8:40 pm,Game Preview and Matchups,Game Preview and Matchups,Game Preview and Matchups,Game Preview and Matchups,Game Preview and Matchups,Game Preview and Matchups,Game Preview and Matchups,Game Preview and Matchups,Game Preview and Matchups,Game Preview and Matchups,NaN,NaN,NaN
159,Thursday Sep 26,COL,H,STL,3:10 pm,3:10 pm,3:10 pm,Game Preview and Matchups,Game Preview and Matchups,Game Preview and Matchups,Game Preview and Matchups,Game Preview and Matchups,Game Preview and Matchups,Game Preview and Matchups,Game Preview and Matchups,Game Preview and Matchups,Game Preview and Matchups,NaN,NaN,NaN
160,Friday Sep 27,COL,H,LAD,8:10 pm,8:10 pm,8:10 pm,Game Preview and Matchups,Game Preview and Matchups,Game Preview and Matchups,Game Preview and Matchups,Game Preview and Matchups,Game Preview and Matchups,Game Preview and Matchups,Game Preview and Matchups,Game Preview and Matchups,Game Preview and Matchups,NaN,NaN,NaN
161,Saturday Sep 28,COL,H,LAD,8:10 pm,8:10 pm,8:10 pm,Game Preview and Matchups,Game Preview and Matchups,Game Preview and Matchups,Game Preview and Matchups,Game Preview and Matchups,Game Preview and Matchups,Game Preview and Matchups,Game Preview and Matchups,Game Preview and Matchups,Game Preview and Matchups,NaN,NaN,NaN
162,Sunday Sep 29,COL,H,LAD,3:10 pm,3:10 pm,3:10 pm,Game Preview and Matchups,Game Preview and Matchups,Game Preview and Matchups,Game Preview and Matchups,Game Preview and Matchups,Game Preview and Matchups,Game Preview and Matchups,Game Preview and Matchups,Game Preview and Matchups,Game Preview and Matchups,NaN,NaN,NaN
